In [77]:
import psycopg2
import pandas as pd 
import numpy as np
from config import params

# conn = psycopg2.connect(servers.params)
conn = psycopg2.connect(**params)

In [3]:
# cursor connection
cursor = conn.cursor()
cursor.execute("SELECT * FROM real_flight WHERE cancelled = 'False' and diverted = 'False'")
rows = cursor.fetchall()
cursor.close()
conn.close()

In [78]:
df = pd.DataFrame(rows, columns=[desc.name for desc in cursor.description])
cleaned_df = df.dropna(subset=['arr_del15', 'dep_del15'])

day_of_month              int64
day_of_week               int64
op_unique_carrier        object
op_carrier_airline_id    object
op_carrier               object
tail_num                 object
op_carrier_fl_num        object
origin_airport_id        object
origin_airport_seq_id    object
origin                   object
dest_airport_id          object
dest_airport_seq_id      object
dest                     object
dep_time                 object
dep_del15                  bool
dep_time_blk             object
arr_time                 object
arr_del15                  bool
cancelled                  bool
diverted                   bool
distance                 object
dtype: object

In [81]:
#Classifying each delayed arilne by labeling 1 if either `ARR_DEL15` or `DEP_DEL15` or 0 if none were delayed
cleaned_df["Delayed"] = int(0)
cleaned_df.loc[cleaned_df["arr_del15"] == True, "Delayed"] = int(1)
cleaned_df.loc[cleaned_df["dep_del15"] == True, "Delayed"] = int(1)


In [93]:
#Creating groups to find the ration per airline of delayed flights
delayed_ratio_airline = cleaned_df.groupby("op_unique_carrier")["Delayed"].value_counts().reset_index(name="ratio")
delayed_ratio_airline["Ratio_Percentage"] = round((delayed_ratio_airline["ratio"] / delayed_ratio_airline.groupby('op_unique_carrier')["ratio"].transform('sum')) * 100,2)
sort_airline_ratio = delayed_ratio_airline.sort_values(by="Ratio_Percentage",ascending=False)
sort_airline_ratio.to_csv('delayed_airlines.csv')

In [100]:
#origin_airport_id
delayed_ratio_airport = cleaned_df.groupby(["origin_airport_id","op_unique_carrier"])["Delayed"].value_counts().reset_index(name="ratio")
delayed_ratio_airport["Ratio_Percentage"] = round((delayed_ratio_airline["ratio"] / delayed_ratio_airline.groupby('op_unique_carrier')["ratio"].transform('sum')) * 100,2)
sort_airport_ratio = delayed_ratio_airport.sort_values(by="Ratio_Percentage",ascending=False)
sort_airport_ratio.to_csv('delayed_airports.csv')


,origin_airport_id,op_unique_carrier,Delayed,ratio,Ratio_Percentage
16,10140,AS,0,38,87.80
0,10135,9E,0,67,85.61
8,10135,OH,0,114,85.57
28,10140,MQ,1,8,85.14
32,10140,UA,1,20,84.93
...,...,...,...,...,...
3519,16218,OO,1,22,NaN
3520,16218,YV,0,117,NaN
3521,16218,YV,1,30,NaN
3522,16869,OO,0,80,NaN
